In [2]:
import tensorflow as tf
def windowed_dataset(series,window_size,batch_size,shuffle_buffer):
  series = tf.expand_dims(series,axis=-1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size+1,shift=1,drop_remainder = True)
  dataset = dataset.flat_map(lambda window:window.batch(window_size+1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window:(window[:-1],window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [4]:
import numpy as np
def trend(time, slope=0):
 return slope * time
def seasonal_pattern(season_time):
 return np.where(season_time < 0.4,
 np.cos(season_time * 2 * np.pi),
 1 / np.exp(3 * season_time))
def seasonality(time, period, amplitude=1, phase=0):
 season_time = ((time + phase) % period) / period
 return amplitude * seasonal_pattern(season_time)
def noise(time, noise_level=1, seed=None):
 rnd = np.random.RandomState(seed)
 return rnd.randn(len(time)) * noise_level
time = np.arange(4 * 365 + 1, dtype="float32")
series = trend(time, 0.1)
baseline = 10
amplitude = 20
slope = 0.09
noise_level = 5
series = baseline + trend(time, slope)
series += seasonality(time, period=365, amplitude=amplitude)
series += noise(time, noise_level, seed=42)
split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000


In [5]:
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
model = tf.keras.models.Sequential([
 tf.keras.layers.Conv1D(filters=128, kernel_size=3,
 strides=1, padding="causal",
 activation="relu",
input_shape=[None, 1]),
 tf.keras.layers.Dense(28, activation="relu"),
 tf.keras.layers.Dense(10, activation="relu"),
 tf.keras.layers.Dense(1),
])
optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.5)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(dataset, epochs=100, verbose=1)


Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


31/31 [==============================] - 1s 14ms/step - loss: 1421.8270
Epoch 2/100
31/31 [==============================] - 0s 3ms/step - loss: 105.7285
Epoch 3/100
31/31 [==============================] - 0s 3ms/step - loss: 83.1833
Epoch 4/100
31/31 [==============================] - 0s 3ms/step - loss: 74.1589
Epoch 5/100
31/31 [==============================] - 0s 3ms/step - loss: 69.7115
Epoch 6/100
31/31 [==============================] - 0s 4ms/step - loss: 67.7478
Epoch 7/100
31/31 [==============================] - 0s 3ms/step - loss: 66.5935
Epoch 8/100
31/31 [==============================] - 0s 4ms/step - loss: 65.8433
Epoch 9/100
31/31 [==============================] - 0s 3ms/step - loss: 65.7479
Epoch 10/100
31/31 [==============================] - 0s 3ms/step - loss: 65.4344
Epoch 11/100
31/31 [==============================] - 0s 3ms/step - loss: 64.7817
Epoch 12/100
31/31 [==============================] - 0s 3ms/step - loss: 64.2646
Epoch 13/100
31/31 [=============

In [6]:
def model_forecast(model, series, window_size):
 ds = tf.data.Dataset.from_tensor_slices(series)
 ds = ds.window(window_size, shift=1, drop_remainder=True)
 ds = ds.flat_map(lambda w: w.batch(window_size))
 ds = ds.batch(32).prefetch(1)
 forecast = model.predict(ds)
 return forecast


In [ ]:
forecast = model_forecast(model, series[..., np.newaxis], window_size)
results = forecast[split_time - window_size:-1, -1, 0]
